In [1]:
import torch
import torchvision
import torchvision.models as models
from torch import nn, optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.torch.cuda.is_available():
    print(" GPU detected : ", torch.cuda.get_device_name(torch.cuda.current_device()))

torch.__version__

'0.4.1'

In [4]:
class ConvNet(nn.Module):
    def __init__(self, pre_trained_model, hidden_layers_size, output_size, verbose):
        super().__init__()
        
        self.pre_trained_model = pre_trained_model
        fc_in = self.pre_trained_model.fc.in_features
        
        #On modifier la couche de prédiction en une couche plus complexe et qui correspond à notre nombre de classes à prédire
        # NOTE : pour certains réseaux la dernière couche ne s'appelle pas fc
        self.pre_trained_model.fc = nn.Sequential(nn.Linear(fc_in, hidden_layers_size),
                                nn.ReLU(),
                                nn.Linear(hidden_layers_size, hidden_layers_size),
                                nn.ReLU(),
                                nn.Linear(hidden_layers_size, output_size))
        
        
        #On freeze le gradient sur n-1 une couche
        nb_layers = sum(1 for x in self.pre_trained_model.children())
        i=0
        for layer in self.pre_trained_model.children():
            if i < nb_layers - 1:
                for param in layer.parameters():
                    param.requires_grad = False
                i+=1
        
        if verbose:
            print("---- Couches du réseau ----")
            for name, child in self.pre_trained_model.named_children():
                for name2, params in child.named_parameters():
                    print(name, name2)
                    print("Freeze : ", not(params.requires_grad))
                print()
                
                 
    def forward(self, x):
        return self.pre_trained_model(x)

In [5]:

convNet = ConvNet(torchvision.models.resnet152(pretrained=True), hidden_layers_size=1024, output_size=16, verbose=True)

if torch.torch.cuda.is_available():
    convNet=convNet.cuda()
    
optimizer = optim.SGD(convNet.parameters(), lr=0.01)

criterion = torch.nn.CrossEntropyLoss()

---- Couches du réseau ----
conv1 weight
Freeze :  True

bn1 weight
Freeze :  True
bn1 bias
Freeze :  True



layer1 0.conv1.weight
Freeze :  True
layer1 0.bn1.weight
Freeze :  True
layer1 0.bn1.bias
Freeze :  True
layer1 0.conv2.weight
Freeze :  True
layer1 0.bn2.weight
Freeze :  True
layer1 0.bn2.bias
Freeze :  True
layer1 0.conv3.weight
Freeze :  True
layer1 0.bn3.weight
Freeze :  True
layer1 0.bn3.bias
Freeze :  True
layer1 0.downsample.0.weight
Freeze :  True
layer1 0.downsample.1.weight
Freeze :  True
layer1 0.downsample.1.bias
Freeze :  True
layer1 1.conv1.weight
Freeze :  True
layer1 1.bn1.weight
Freeze :  True
layer1 1.bn1.bias
Freeze :  True
layer1 1.conv2.weight
Freeze :  True
layer1 1.bn2.weight
Freeze :  True
layer1 1.bn2.bias
Freeze :  True
layer1 1.conv3.weight
Freeze :  True
layer1 1.bn3.weight
Freeze :  True
layer1 1.bn3.bias
Freeze :  True
layer1 2.conv1.weight
Freeze :  True
layer1 2.bn1.weight
Freeze :  True
layer1 2.bn1.bias
Freeze :  True
layer1 2.conv2.weight
Fre